In [18]:
import tensorflow as tf
import numpy as np
import math

import os
import random
import sklearn

words = {}

def load_input_sentences(filepath):
    sentence_file = open(filepath,'r')
    sentence = sentence_file.readline()
    sentences = []
    solutions = []
    while sentence != '':
        substrings = sentence.split(' ')
        solution = substrings[-1].strip()
        sentence = ''
        for i in range(len(substrings)-1):
            sentence = sentence + substrings[i]+' ' 
        solutions.append(int(solution))
        sentences.append(sentence.strip())
        sentence = sentence_file.readline()
        
    return sentences, solutions


def create_vocabulary():
    num = 0
    vocab_size = 0
    for sentence in sentences:
        substrings = sentence.split(' ')
        for string in substrings:
            if words.get(string) == None:
                #print num
                words[string] = num
                num += 1
                vocab_size = num
    
    return vocab_size
    

def fc(input, num_output, name = 'fc'):
    with tf.variable_scope(name):
        return tf.layers.dense(input, num_output)
        # num_input = input.get_shape()[1]
        # W = tf.get_variable('w', [num_input, num_output], tf.float32, tf.random_normal_initializer(0.0, 0.2))
        # b = tf.get_variable('b', [num_output], initializer = tf.random_normal_initializer(0.0, 0.02))
        # return tf.matmul(input, W) + b

In [19]:
sentences, solutions = load_input_sentences('sentences_nlp.txt')
sentences_test, solutions_test = load_input_sentences('sentences_nlp_test.txt')
vocab_size = create_vocabulary()
print vocab_size
# test_sentences, test_solutions = load_input_sentences('sentences_nlp_test.txt')

23


In [23]:
def batch_norm(input, is_training):
    out = tf.contrib.layers.batch_norm(input, decay = 0.99, center = True, scale = True,
                                       is_training = is_training, updates_collections = None)
    return out

def leaky_relu(input, alpha = 0.2):
    return tf.maximum(alpha * input, input)


class Text_Arithematic(object):

    def __init__(self):
        self.num_epoch = 25
        self.batch_size = 32
        self.log_step = 50
        self.learning_rate = 1e-4
        
        self._net_called = False


        self.real_sentences = tf.placeholder(tf.float32, [None, vocab_size])
        
        self.real_labels = tf.placeholder(tf.float32, [None, 100])
        self.labels = tf.placeholder(tf.int64, [None])
        self.is_train = tf.placeholder(tf.bool)
        
        self._init_ops()

    def _network(self,text_embedding, is_train):
        with tf.variable_scope('net', reuse = self._net_called):
            self._net_called = True
            fc1 = fc(text_embedding, 256, 'fc1')
            relu1 = leaky_relu(fc1)
            #batch1 = batch_norm(relu1,is_train)
            
            fc2 = fc(relu1, 512, 'fc2')
            relu2 = leaky_relu(fc2)
            #batch2 = batch_norm(relu2,is_train)
            
            fc3 = fc(relu2, 1024, 'fc3')
            relu3 = leaky_relu(fc3)
            dropout3 = tf.nn.dropout(relu3, 0.5)
            
            fc4 = fc(dropout3, 2048, 'fc4')
            relu4 = leaky_relu(fc4)
            dropout4 = tf.nn.dropout(relu4, 0.5)
            
            fc5 = fc(dropout4, 2048, 'fc5')
            relu5 = leaky_relu(fc5)
            dropout5 = tf.nn.dropout(relu5, 0.5)
            
            fc6 = fc(dropout5, 1024, 'fc6')
            relu6 = leaky_relu(fc6)
            dropout6 = tf.nn.dropout(relu6, 0.5)
            
            fc7 = fc(dropout6, 512, 'fc7')
            relu7 = leaky_relu(fc7)
            dropout7 = tf.nn.dropout(relu7, 0.5)
            
            fc8 = fc(dropout7, 256, 'fc8')
            relu8 = leaky_relu(fc8)
            
            fc9 = fc(relu8, 100, 'fc9')
            relu9 = leaky_relu(fc9)
            
            sigmoid6 = tf.sigmoid(relu9)
            print(sigmoid6.shape)
            return relu9

    def _loss(self, labels, logits):
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = labels, logits = logits)
        return tf.reduce_mean(loss)
    
    # Define operations
    def _init_ops(self):
        self.generated_op = self._network(self.real_sentences, False)
        self.loss_op = self._loss(self.real_labels, self._network(self.real_sentences, self.is_train))
        
        logits = self._network(self.real_sentences, self.is_train)
        print(logits.shape)
        
        predict = tf.argmax(logits, 1)
        #print (predict.shape, self.labels.shape)
        correct = tf.equal(predict, self.labels)
        self.accuracy_op = tf.reduce_mean(tf.cast(correct, tf.float32))
        
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "net")
        self.train_op = optimizer.minimize(self.loss_op,var_list = train_vars)
        
        
    # Training function
    def train(self, sess,train_sentences,train_labels ):
        sess.run(tf.global_variables_initializer())

        num_train = train_sentences.shape[0]
        step = 0
        
        dis_losses = []
        gen_losses = []
        for epoch in range(self.num_epoch):
            for i in range(num_train // (self.batch_size)):
                step += 1
                sentence_batch = train_sentences[i * self.batch_size : (i + 1) * self.batch_size]
                labels = np.zeros([self.batch_size, 100])
                label = []
                embeddings = np.zeros([self.batch_size, vocab_size])
                for k in range(len(sentence_batch)):
                    sentence = sentence_batch[k]
                    substrings = sentence.split(' ') 
                    for string in substrings:
                        index = words.get(string)
                        embeddings[k][index] = 1
                
                for j in range(self.batch_size ):
                    labels[j][train_labels[i*self.batch_size + j]] = 1
                    label.append(train_labels[i*self.batch_size + j])
                
                
                feed_dict = { self.real_sentences:embeddings, self.real_labels:labels,self.is_train:True, self.labels:label}

                _, loss, accuracy = sess.run([self.train_op, self.loss_op, self.accuracy_op], feed_dict = feed_dict)

                if step % self.log_step == 0:
                    print('Iteration {0}: loss = {1:.4f}, accuracy = {2:.4f}'.format(step,loss, accuracy))
            self.generate_one_sample()
            
            


    # Generates a single sample from input code
    def generate_one_sample(self):
        count = 1000
        sentence_batch = sentences_test[:count]
        solutions_batch = solutions_test[:count]
        
        test_size = count
        
        labels = np.zeros([test_size, 100])
        label = []
        embeddings = np.zeros([test_size, vocab_size])
        for i in range(len(sentence_batch)):
            sentence = sentence_batch[i]
            substrings = sentence.split(' ') 
            for string in substrings:
                index = words.get(string)
                embeddings[i][index] = 1
                
        gen_vis_feed_dict = { self.real_sentences:embeddings, self.is_train : False}       
        generated = sess.run(self.generated_op, feed_dict = gen_vis_feed_dict)
        predict = tf.argmax(generated, 1)
        correct = 0
        for i in range(count):
            if predict[i].eval() == solutions_batch[i]:
                correct += 1
        acc = float(correct) / float(count)
        print "Currently, the testing accuracy is: ", acc

    
tf.reset_default_graph()

with tf.Session() as sess:
    dcgan = Text_Arithematic()
    sess.run(tf.global_variables_initializer())
    train_sentences = np.array(sentences)
    train_labels = np.array(solutions)
    dcgan.train(sess,train_sentences,train_labels)
    var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'net')
    saver = tf.train.Saver(var_list)
    saver.save(sess, 'model/dcgan')



(?, 100)
(?, 100)
(?, 100)
(?, 100)
Iteration 50: loss = 0.0875, accuracy = 0.0000
Iteration 100: loss = 0.0530, accuracy = 0.0312
Iteration 150: loss = 0.0513, accuracy = 0.0000
Iteration 200: loss = 0.0521, accuracy = 0.0312
Iteration 250: loss = 0.0503, accuracy = 0.0312
Iteration 300: loss = 0.0483, accuracy = 0.0000
Iteration 350: loss = 0.0460, accuracy = 0.0625
Iteration 400: loss = 0.0463, accuracy = 0.0312
Iteration 450: loss = 0.0466, accuracy = 0.1562
Iteration 500: loss = 0.0441, accuracy = 0.0625
Iteration 550: loss = 0.0463, accuracy = 0.0938
Iteration 600: loss = 0.0411, accuracy = 0.1250
Iteration 650: loss = 0.0455, accuracy = 0.0625
Iteration 700: loss = 0.0447, accuracy = 0.1562
Iteration 750: loss = 0.0443, accuracy = 0.0625
Iteration 800: loss = 0.0417, accuracy = 0.0625
Iteration 850: loss = 0.0453, accuracy = 0.1250
Iteration 900: loss = 0.0421, accuracy = 0.0938
Iteration 950: loss = 0.0426, accuracy = 0.0938
Iteration 1000: loss = 0.0384, accuracy = 0.1250
Iter

Iteration 8250: loss = 0.0233, accuracy = 0.3750
Iteration 8300: loss = 0.0192, accuracy = 0.5312
Iteration 8350: loss = 0.0144, accuracy = 0.6250
Iteration 8400: loss = 0.0138, accuracy = 0.6562
Iteration 8450: loss = 0.0160, accuracy = 0.6875
Iteration 8500: loss = 0.0145, accuracy = 0.6250
Iteration 8550: loss = 0.0224, accuracy = 0.4375
Iteration 8600: loss = 0.0210, accuracy = 0.4375
Iteration 8650: loss = 0.0167, accuracy = 0.5312
Iteration 8700: loss = 0.0237, accuracy = 0.3750
Iteration 8750: loss = 0.0161, accuracy = 0.6250
Iteration 8800: loss = 0.0189, accuracy = 0.5000
Iteration 8850: loss = 0.0177, accuracy = 0.5312
Iteration 8900: loss = 0.0206, accuracy = 0.4375
Iteration 8950: loss = 0.0200, accuracy = 0.4375
Iteration 9000: loss = 0.0165, accuracy = 0.5625
Iteration 9050: loss = 0.0126, accuracy = 0.7812
Iteration 9100: loss = 0.0169, accuracy = 0.6250
Iteration 9150: loss = 0.0203, accuracy = 0.5000
Iteration 9200: loss = 0.0198, accuracy = 0.4375
Iteration 9250: loss

KeyboardInterrupt: 